# Stochastic Algorithms

## Adaptive Random Search

In [11]:
def objective_function(vector)
  return vector.inject(0) {|sum, x| sum +  (x ** 2.0)}
end

:objective_function

In [12]:
def rand_in_bounds(min, max)
  return min + ((max-min) * rand()) 
end

:rand_in_bounds

In [13]:
def random_vector(minmax)
  return Array.new(minmax.size) do |i|      
    rand_in_bounds(minmax[i][0], minmax[i][1])
  end
end

:random_vector

In [14]:
def take_step(minmax, current, step_size)
  position = Array.new(current.size)
  position.size.times do |i|
    min = [minmax[i][0], current[i]-step_size].max
    max = [minmax[i][1], current[i]+step_size].min
    position[i] = rand_in_bounds(min, max)
  end
  return position
end

:take_step

In [15]:
def large_step_size(iter, step_size, s_factor, l_factor, iter_mult)
  return step_size * l_factor if iter>0 and iter.modulo(iter_mult) == 0
  return step_size * s_factor
end

:large_step_size

In [16]:
def take_steps(bounds, current, step_size, big_stepsize)
  step, big_step = {}, {}
  step[:vector] = take_step(bounds, current[:vector], step_size)
  step[:cost] = objective_function(step[:vector])
  big_step[:vector] = take_step(bounds,current[:vector],big_stepsize)
  big_step[:cost] = objective_function(big_step[:vector])    
  return step, big_step
end

:take_steps

In [17]:
def search(max_iter, bounds, init_factor, s_factor, l_factor, iter_mult, max_no_impr)
  step_size = (bounds[0][1]-bounds[0][0]) * init_factor
  current, count = {}, 0
  current[:vector] = random_vector(bounds)
  current[:cost] = objective_function(current[:vector])
  max_iter.times do |iter|
    big_stepsize = large_step_size(iter, step_size, s_factor, l_factor, iter_mult)
    step, big_step = take_steps(bounds, current, step_size, big_stepsize)
    if step[:cost] <= current[:cost] or big_step[:cost] <= current[:cost]
      if big_step[:cost] <= step[:cost]
        step_size, current = big_stepsize, big_step
      else
        current = step
      end
      count = 0
    else
      count += 1
      count, step_size = 0, (step_size/s_factor) if count >= max_no_impr
    end
    #puts " > iteration #{(iter+1)}, best=#{current[:cost]}"
  end
  return current
end

:search

In [18]:
# problem configuration
problem_size = 2
bounds = Array.new(problem_size) {|i| [-5, +5]}

# algorithm configuration
max_iter = 1000
init_factor = 0.05
s_factor = 1.3
l_factor = 3.0
iter_mult = 10
max_no_impr = 30

# execute the algorithm
best = search(max_iter, bounds, init_factor, s_factor, l_factor, iter_mult, max_no_impr)
puts "Done. Best Solution:","c=#{best[:cost]}","v=#{best[:vector].inspect}"


Done. Best Solution:
c=1.4887793569443956e-08
v=[0.00011064396661582265, -5.1436428928928156e-05]


## Greedy Randomized Adaptive Search Procedure (GRASP)

In [19]:
def euc_2d(c1, c2)
  Math.sqrt((c1[0] - c2[0])**2.0 + (c1[1] - c2[1])**2.0).round
end

:euc_2d

In [20]:
def cost(perm, cities)
  distance =0
  perm.each_with_index do |c1, i|
    c2 = (i==perm.size-1) ? perm[0] : perm[i+1]
    distance += euc_2d(cities[c1], cities[c2])
  end
  return distance
end

:cost

In [21]:
def stochastic_two_opt(permutation)
  perm = Array.new(permutation)
  c1, c2 = rand(perm.size), rand(perm.size)
  exclude = [c1]
  exclude << ((c1==0) ? perm.size-1 : c1-1)
  exclude << ((c1==perm.size-1) ? 0 : c1+1)
  c2 = rand(perm.size) while exclude.include?(c2)
  c1, c2 = c2, c1 if c2 < c1
  perm[c1...c2] = perm[c1...c2].reverse
  return perm
end

:stochastic_two_opt

In [22]:
def local_search(best, cities, max_no_improv)
  count = 0
  begin
    candidate = {:vector=>stochastic_two_opt(best[:vector])}
    candidate[:cost] = cost(candidate[:vector], cities)    
    count = (candidate[:cost] < best[:cost]) ? 0 : count+1
    best = candidate if candidate[:cost] < best[:cost]    
  end until count >= max_no_improv
  return best
end

:local_search

In [23]:
def construct_randomized_greedy_solution(cities, alpha)
  candidate = {}
  candidate[:vector] = [rand(cities.size)]
  allCities = Array.new(cities.size) {|i| i}
  while candidate[:vector].size < cities.size
    candidates = allCities - candidate[:vector]
    costs = Array.new(candidates.size) do |i| 
      euc_2d(cities[candidate[:vector].last], cities[i])
    end
    rcl, max, min = [], costs.max, costs.min
    costs.each_with_index do |c,i| 
      rcl << candidates[i] if c <= (min + alpha*(max-min))
    end
    candidate[:vector] << rcl[rand(rcl.size)]
  end
  candidate[:cost] = cost(candidate[:vector], cities)
  return candidate
end

:construct_randomized_greedy_solution

In [24]:
def search(cities, max_iter, max_no_improv, alpha)
  best = nil
  max_iter.times do |iter|
    candidate = construct_randomized_greedy_solution(cities, alpha);
    candidate = local_search(candidate, cities, max_no_improv)
    best = candidate if best.nil? or candidate[:cost] < best[:cost]
    #puts " > iteration #{(iter+1)}, best=#{best[:cost]}"
  end
  return best
end

:search

In [25]:
# problem configuration
berlin52 = [
  [565,575],[25,185],[345,750],[945,685],[845,655],
  [880,660],[25,230],[525,1000],[580,1175],[650,1130],[1605,620],
  [1220,580],[1465,200],[1530,5],[845,680],[725,370],[145,665],
  [415,635],[510,875],[560,365],[300,465],[520,585],[480,415],
  [835,625],[975,580],[1215,245],[1320,315],[1250,400],[660,180],
  [410,250],[420,555],[575,665],[1150,1160],[700,580],[685,595],
  [685,610],[770,610],[795,645],[720,635],[760,650],[475,960],
  [95,260],[875,920],[700,500],[555,815],[830,485],[1170,65],
  [830,610],[605,625],[595,360],[1340,725],[1740,245]]
  
# algorithm configuration
max_iter = 50
max_no_improv = 50
greediness_factor = 0.3

# execute the algorithm
best = search(berlin52, max_iter, max_no_improv, greediness_factor)
puts "Done. Best Solution:","c=#{best[:cost]}","v=#{best[:vector].inspect}"


Done. Best Solution:
c=9994
v=[23, 37, 39, 36, 33, 47, 5, 4, 42, 18, 2, 17, 30, 21, 19, 22, 20, 16, 6, 41, 1, 29, 28, 15, 14, 3, 24, 27, 11, 25, 46, 26, 12, 13, 51, 10, 50, 32, 9, 8, 7, 40, 44, 38, 31, 35, 34, 0, 48, 49, 43, 45]


## Guided Local Search

In [26]:
def euc_2d(c1, c2)
  Math.sqrt((c1[0] - c2[0])**2.0 + (c1[1] - c2[1])**2.0).round
end

:euc_2d

In [27]:
def random_permutation(cities)
  perm = Array.new(cities.size){|i| i}
  perm.each_index do |i|
    r = rand(perm.size-i) + i
    perm[r], perm[i] = perm[i], perm[r]
  end
  return perm
end

:random_permutation

In [28]:
def stochastic_two_opt(permutation)
  perm = Array.new(permutation)
  c1, c2 = rand(perm.size), rand(perm.size)
  exclude = [c1]
  exclude << ((c1==0) ? perm.size-1 : c1-1)
  exclude << ((c1==perm.size-1) ? 0 : c1+1)
  c2 = rand(perm.size) while exclude.include?(c2)
  c1, c2 = c2, c1 if c2 < c1
  perm[c1...c2] = perm[c1...c2].reverse
  return perm
end

:stochastic_two_opt

In [29]:
def augmented_cost(permutation, penalties, cities, lambda)
  distance, augmented = 0, 0
  permutation.each_with_index do |c1, i|
    c2 = (i==permutation.size-1) ? permutation[0] : permutation[i+1]
    c1, c2 = c2, c1 if c2 < c1
    d = euc_2d(cities[c1], cities[c2])
    distance += d
    augmented += d + (lambda * (penalties[c1][c2]))
  end
  return [distance, augmented]
end

:augmented_cost

In [30]:
def cost(cand, penalties, cities, lambda)
  cost, acost = augmented_cost(cand[:vector], penalties, cities, lambda)
  cand[:cost], cand[:aug_cost] = cost, acost
end

:cost

In [31]:
def local_search(current, cities, penalties, max_no_improv, lambda)
  cost(current, penalties, cities, lambda)
  count = 0
  begin
    candidate = {:vector=> stochastic_two_opt(current[:vector])}
    cost(candidate, penalties, cities, lambda)
    count = (candidate[:aug_cost] < current[:aug_cost]) ? 0 : count+1
    current = candidate if candidate[:aug_cost] < current[:aug_cost]
  end until count >= max_no_improv
  return current
end

:local_search

In [32]:
def calculate_feature_utilities(penal, cities, permutation)
  utilities = Array.new(permutation.size,0)
  permutation.each_with_index do |c1, i|
    c2 = (i==permutation.size-1) ? permutation[0] : permutation[i+1]
    c1, c2 = c2, c1 if c2 < c1
    utilities[i] = euc_2d(cities[c1], cities[c2]) / (1.0 + penal[c1][c2])
  end
  return utilities
end

:calculate_feature_utilities

In [33]:
def update_penalties!(penalties, cities, permutation, utilities)
  max = utilities.max()
  permutation.each_with_index do |c1, i|
    c2 = (i==permutation.size-1) ? permutation[0] : permutation[i+1]
    c1, c2 = c2, c1 if c2 < c1
    penalties[c1][c2] += 1 if utilities[i] == max
  end
  return penalties
end

:update_penalties!

In [36]:
def search(max_iterations, cities, max_no_improv, lambda)
  current = {:vector=>random_permutation(cities)}
  best = nil
  penalties = Array.new(cities.size){ Array.new(cities.size, 0) }
  max_iterations.times do |iter|
    current=local_search(current, cities, penalties, max_no_improv, lambda)
    utilities=calculate_feature_utilities(penalties,cities,current[:vector])
    update_penalties!(penalties, cities, current[:vector], utilities)
    best = current if best.nil? or current[:cost] < best[:cost]
    #puts " > iter=#{(iter+1)}, best=#{best[:cost]}, aug=#{best[:aug_cost]}"
  end
  return best
end

:search

In [37]:
# problem configuration
# algorithm configuration
max_iterations = 150
max_no_improv = 20
alpha = 0.3
local_search_optima = 12000.0
lambda = alpha * (local_search_optima/berlin52.size.to_f)

# execute the algorithm
best = search(max_iterations, berlin52, max_no_improv, lambda)
puts "Done. Best Solution:","c=#{best[:cost]}","v=#{best[:vector].inspect}"

Done. Best Solution:
c=8847
v=[33, 43, 15, 45, 38, 39, 37, 36, 44, 18, 40, 7, 8, 9, 32, 42, 14, 4, 23, 47, 5, 3, 24, 11, 50, 10, 51, 13, 26, 27, 25, 12, 46, 28, 49, 19, 22, 17, 30, 20, 29, 1, 6, 41, 16, 2, 21, 31, 0, 48, 35, 34]


## Iterated Local Search

In [38]:
def euc_2d(c1, c2)
  Math.sqrt((c1[0] - c2[0])**2.0 + (c1[1] - c2[1])**2.0).round
end

:euc_2d

In [39]:
def cost(permutation, cities)
  distance =0
  permutation.each_with_index do |c1, i|
    c2 = (i==permutation.size-1) ? permutation[0] : permutation[i+1]
    distance += euc_2d(cities[c1], cities[c2])
  end
  return distance
end

:cost

In [40]:
def random_permutation(cities)
  perm = Array.new(cities.size){|i| i}
  perm.each_index do |i|
    r = rand(perm.size-i) + i
    perm[r], perm[i] = perm[i], perm[r]
  end
  return perm
end

:random_permutation

In [41]:
def stochastic_two_opt(permutation)
  perm = Array.new(permutation)
  c1, c2 = rand(perm.size), rand(perm.size)
  exclude = [c1]
  exclude << ((c1==0) ? perm.size-1 : c1-1)
  exclude << ((c1==perm.size-1) ? 0 : c1+1)
  c2 = rand(perm.size) while exclude.include?(c2)
  c1, c2 = c2, c1 if c2 < c1
  perm[c1...c2] = perm[c1...c2].reverse
  return perm
end

:stochastic_two_opt

In [42]:
def local_search(best, cities, max_no_improv)
  count = 0
  begin
    candidate = {:vector=>stochastic_two_opt(best[:vector])}
    candidate[:cost] = cost(candidate[:vector], cities)    
    count = (candidate[:cost] < best[:cost]) ? 0 : count+1
    best = candidate if candidate[:cost] < best[:cost]    
  end until count >= max_no_improv
  return best
end

:local_search

In [43]:
def double_bridge_move(perm)
  pos1 = 1 + rand(perm.size / 4)
  pos2 = pos1 + 1 + rand(perm.size / 4)
  pos3 = pos2 + 1 + rand(perm.size / 4)
  p1 = perm[0...pos1] + perm[pos3..perm.size]
  p2 = perm[pos2...pos3] + perm[pos1...pos2]
  return p1 + p2
end

:double_bridge_move

In [44]:
def perturbation(cities, best)
  candidate = {}
  candidate[:vector] = double_bridge_move(best[:vector])
  candidate[:cost] = cost(candidate[:vector], cities)
  return candidate
end

:perturbation

In [45]:
def search(cities, max_iterations, max_no_improv)
  best = {}
  best[:vector] = random_permutation(cities)
  best[:cost] = cost(best[:vector], cities)
  best = local_search(best, cities, max_no_improv)
  max_iterations.times do |iter|
    candidate = perturbation(cities, best)
    candidate = local_search(candidate, cities, max_no_improv)
    best = candidate if candidate[:cost] < best[:cost]
    #puts " > iteration #{(iter+1)}, best=#{best[:cost]}"
  end
  return best
end

:search

In [46]:
# algorithm configuration
max_iterations = 100
max_no_improv = 50

# execute the algorithm
best = search(berlin52, max_iterations, max_no_improv)
puts "Done. Best Solution:","c=#{best[:cost]}","v=#{best[:vector].inspect}"


Done. Best Solution:
c=9078
v=[48, 9, 8, 7, 40, 18, 44, 31, 0, 21, 33, 34, 35, 43, 22, 30, 17, 2, 16, 20, 41, 1, 6, 29, 49, 19, 15, 28, 46, 13, 51, 10, 12, 25, 26, 27, 11, 50, 32, 42, 5, 3, 24, 45, 47, 23, 4, 14, 37, 38, 39, 36]


## Random Search

In [47]:
def objective_function(vector)
  return vector.inject(0) {|sum, x| sum + (x ** 2.0)}
end

:objective_function

In [48]:
def random_vector(minmax)
  return Array.new(minmax.size) do |i|      
    minmax[i][0] + ((minmax[i][1] - minmax[i][0]) * rand())
  end
end

:random_vector

In [49]:
def search(search_space, max_iter)
  best = nil
  max_iter.times do |iter|
    candidate = {}
    candidate[:vector] = random_vector(search_space)
    candidate[:cost] = objective_function(candidate[:vector])
    best = candidate if best.nil? or candidate[:cost] < best[:cost]
    #puts " > iteration=#{(iter+1)}, best=#{best[:cost]}"
  end
  return best
end

:search

In [51]:
# problem configuration
problem_size = 2
search_space = Array.new(problem_size) {|i| [-5, +5]}

# algorithm configuration
max_iter = 100

# execute the algorithm
best = search(search_space, max_iter)
puts "Done. Best Solution:","c=#{best[:cost]}","v=#{best[:vector].inspect}"

Done. Best Solution:
c=0.3172062147005273
v=[-0.4724191948354868, -0.3066371129715346]


## Reactive Tabu Search

In [52]:
def euc_2d(c1, c2)
  Math.sqrt((c1[0] - c2[0])**2.0 + (c1[1] - c2[1])**2.0).round
end

:euc_2d

In [53]:
def cost(perm, cities)
  distance = 0
  perm.each_with_index do |c1, i|
    c2 = (i==perm.size-1) ? perm[0] : perm[i+1]
    distance += euc_2d(cities[c1], cities[c2])
  end
  return distance
end

:cost

In [54]:
def random_permutation(cities)
  perm = Array.new(cities.size){|i| i}
  perm.each_index do |i|
    r = rand(perm.size-i) + i
    perm[r], perm[i] = perm[i], perm[r]
  end
  return perm
end

:random_permutation

In [55]:
def stochastic_two_opt(parent)
  perm = Array.new(parent)
  c1, c2 = rand(perm.size), rand(perm.size)
  exclude = [c1]
  exclude << ((c1==0) ? perm.size-1 : c1-1)
  exclude << ((c1==perm.size-1) ? 0 : c1+1)
  c2 = rand(perm.size) while exclude.include?(c2)
  c1, c2 = c2, c1 if c2 < c1
  perm[c1...c2] = perm[c1...c2].reverse
  return perm, [[parent[c1-1], parent[c1]], [parent[c2-1], parent[c2]]]
end

:stochastic_two_opt

In [56]:
def is_tabu?(edge, tabu_list, iter, prohib_period)
  tabu_list.each do |entry|
    if entry[:edge] == edge
      return true if entry[:iter] >= iter-prohib_period
      return false
    end
  end
  return false
end

:is_tabu?

In [57]:
def make_tabu(tabu_list, edge, iter)
  tabu_list.each do |entry|
    if entry[:edge] == edge
      entry[:iter] = iter
      return entry
    end
  end
  entry = {:edge=>edge, :iter=>iter}
  tabu_list.push(entry)
  return entry
end

:make_tabu

In [58]:
def to_edge_list(perm)
  list = []
  perm.each_with_index do |c1, i|
    c2 = (i==perm.size-1) ? perm[0] : perm[i+1]
    c1, c2 = c2, c1 if c1 > c2
    list << [c1, c2]
  end
  return list
end

:to_edge_list

In [60]:
def equivalent?(el1, el2)
  el1.each {|e| return false if !el2.include?(e) }
  return true
end

:equivalent?

In [61]:
def generate_candidate(best, cities)
  candidate = {}
  candidate[:vector], edges = stochastic_two_opt(best[:vector])
  candidate[:cost] = cost(candidate[:vector], cities)
  return candidate, edges
end

:generate_candidate

In [62]:
def get_candidate_entry(visited_list, permutation)
  edgeList = to_edge_list(permutation)
  visited_list.each do |entry|
    return entry if equivalent?(edgeList, entry[:edgelist])
  end
  return nil
end

:get_candidate_entry

In [63]:
def store_permutation(visited_list, permutation, iteration)
  entry = {}
  entry[:edgelist] = to_edge_list(permutation)
  entry[:iter] = iteration
  entry[:visits] = 1
  visited_list.push(entry)
  return entry
end

:store_permutation

In [64]:
def sort_neighborhood(candidates, tabu_list, prohib_period, iteration)
  tabu, admissable = [], []
  candidates.each do |a|
    if is_tabu?(a[1][0], tabu_list, iteration, prohib_period) or
       is_tabu?(a[1][1], tabu_list, iteration, prohib_period)
      tabu << a
    else
      admissable << a
    end
  end
  return [tabu, admissable]
end

:sort_neighborhood

In [65]:
def search(cities, max_cand, max_iter, increase, decrease)
  current = {:vector=>random_permutation(cities)}
  current[:cost] = cost(current[:vector], cities)
  best = current
  tabu_list, prohib_period = [], 1
  visited_list, avg_size, last_change = [], 1, 0

  max_iter.times do |iter|
    candidate_entry = get_candidate_entry(visited_list, current[:vector])

    if !candidate_entry.nil?
      repetition_interval = iter - candidate_entry[:iter]
      candidate_entry[:iter] = iter
      candidate_entry[:visits] += 1

      if repetition_interval < 2*(cities.size-1)
        avg_size = 0.1*(iter-candidate_entry[:iter]) + 0.9*avg_size
        prohib_period = (prohib_period.to_f * increase)
        last_change = iter
      end
    
    else
      store_permutation(visited_list, current[:vector], iter)
    end
    
    if iter-last_change > avg_size
      prohib_period = [prohib_period*decrease,1].max
      last_change = iter
    end
    
    candidates = Array.new(max_cand) do |i| 
      generate_candidate(current, cities)
    end
    
    candidates.sort! {|x,y| x.first[:cost] <=> y.first[:cost]}        
    tabu,admis = sort_neighborhood(candidates,tabu_list,prohib_period,iter)
    
    if admis.size < 2
      prohib_period = cities.size-2
      last_change = iter
    end
    
    current,best_move_edges = (admis.empty?) ? tabu.first : admis.first
    
    if !tabu.empty? 
      tf = tabu.first[0]
      if tf[:cost]<best[:cost] and tf[:cost]<current[:cost]
        current, best_move_edges = tabu.first
      end
    end
    
    best_move_edges.each {|edge| make_tabu(tabu_list, edge, iter)}
    best = candidates.first[0] if candidates.first[0][:cost] < best[:cost]
    #puts " > it=#{iter}, tenure=#{prohib_period.round}, best=#{best[:cost]}"
  end
  return best
end

:search

In [66]:
# problem configuration
# algorithm configuration
max_iter = 100
max_candidates = 50
increase = 1.3
decrease = 0.9

# execute the algorithm
best = search(berlin52, max_candidates, max_iter, increase, decrease)
puts "Done. Best Solution:","c=#{best[:cost]}","v=#{best[:vector].inspect}"


Done. Best Solution:
c=9047
v=[28, 29, 41, 1, 6, 20, 17, 16, 2, 44, 18, 40, 7, 9, 8, 32, 42, 31, 34, 35, 38, 37, 4, 39, 33, 36, 47, 23, 14, 5, 24, 3, 11, 50, 10, 51, 13, 12, 46, 27, 26, 25, 45, 15, 43, 48, 21, 0, 30, 22, 19, 49]


## Scatter Search

In [68]:
def objective_function(vector)
  return vector.inject(0) {|sum, x| sum +  (x ** 2.0)}
end

:objective_function

In [69]:
def rand_in_bounds(min, max)
  return min + ((max-min) * rand()) 
end

:rand_in_bounds

In [70]:
def random_vector(minmax)
  return Array.new(minmax.size) do |i|      
    rand_in_bounds(minmax[i][0], minmax[i][1])
  end
end

:random_vector

In [71]:
def take_step(minmax, current, step_size)
  position = Array.new(current.size)
  position.size.times do |i|
    min = [minmax[i][0], current[i]-step_size].max
    max = [minmax[i][1], current[i]+step_size].min
    position[i] = rand_in_bounds(min, max)
  end
  return position
end

:take_step

In [72]:
def local_search(best, bounds, max_no_improv, step_size)
  count = 0
  begin
    candidate = {:vector=>take_step(bounds, best[:vector], step_size)}
    candidate[:cost] = objective_function(candidate[:vector])
    count = (candidate[:cost] < best[:cost]) ? 0 : count+1
    best = candidate if candidate[:cost] < best[:cost]    
  end until count >= max_no_improv
  return best
end

:local_search

In [73]:
def construct_initial_set(bounds, set_size, max_no_improv, step_size)
  diverse_set = []
  begin
    cand = {:vector=>random_vector(bounds)}
    cand[:cost] = objective_function(cand[:vector])
    cand = local_search(cand, bounds, max_no_improv, step_size)
    diverse_set << cand if !diverse_set.any? {|x| x[:vector]==cand[:vector]}
  end until diverse_set.size == set_size
  return diverse_set
end

:construct_initial_set

In [74]:
def euclidean_distance(c1, c2)
  sum = 0.0
  c1.each_index {|i| sum += (c1[i]-c2[i])**2.0}  
  return Math.sqrt(sum)
end

:euclidean_distance

In [75]:
def distance(v, set)
  return set.inject(0){|s,x| s + euclidean_distance(v, x[:vector])}
end

:distance

In [76]:
def diversify(diverse_set, num_elite, ref_set_size)
  diverse_set.sort!{|x,y| x[:cost] <=> y[:cost]}
  ref_set = Array.new(num_elite){|i| diverse_set[i]}
  remainder = diverse_set - ref_set
  remainder.each{|c| c[:dist] = distance(c[:vector], ref_set)}
  remainder.sort!{|x,y| y[:dist]<=>x[:dist]}
  ref_set = ref_set + remainder.first(ref_set_size-ref_set.size)
  return [ref_set, ref_set[0]]
end

:diversify

In [77]:
def select_subsets(ref_set)
  additions = ref_set.select{|c| c[:new]}
  remainder = ref_set - additions
  remainder = additions if remainder.nil? or remainder.empty?
  subsets = []
  additions.each do |a| 
    remainder.each{|r| subsets << [a,r] if a!=r && !subsets.include?([r,a])}
  end
  return subsets
end

:select_subsets

In [78]:
def recombine(subset, minmax)
  a, b = subset
  d = Array.new(a[:vector].size) {|i|(b[:vector][i]-a[:vector][i])/2.0}
  children = []
  subset.each do |p|
    direction, r = ((rand<0.5) ? +1.0 : -1.0), rand
    child = {:vector=>Array.new(minmax.size)}
    child[:vector].each_index do |i|
      child[:vector][i] = p[:vector][i] + (direction * r * d[i])
      child[:vector][i]=minmax[i][0] if child[:vector][i]<minmax[i][0]
      child[:vector][i]=minmax[i][1] if child[:vector][i]>minmax[i][1]
    end
    child[:cost] = objective_function(child[:vector])
    children << child
  end
  return children
end

:recombine

In [79]:
def explore_subsets(bounds, ref_set, max_no_improv, step_size)
  was_change = false
  subsets = select_subsets(ref_set)
  ref_set.each{|c| c[:new] = false}
  subsets.each do |subset|
    candidates = recombine(subset, bounds)
    improved = Array.new(candidates.size) do |i| 
      local_search(candidates[i], bounds, max_no_improv, step_size)
    end
    improved.each do |c|
      if !ref_set.any? {|x| x[:vector]==c[:vector]}
        c[:new] = true
        ref_set.sort!{|x,y| x[:cost] <=> y[:cost]}
        if c[:cost] < ref_set.last[:cost]
          ref_set.delete(ref_set.last)
          ref_set << c
          #puts "  >> added, cost=#{c[:cost]}"
          was_change = true
        end
      end
    end
  end
  return was_change
end

:explore_subsets

In [80]:
def search(bounds, max_iter, ref_set_size, div_set_size, max_no_improv, step_size, max_elite)
  diverse_set = construct_initial_set(bounds, div_set_size, max_no_improv, step_size)
  ref_set, best = diversify(diverse_set, max_elite, ref_set_size)
  ref_set.each{|c| c[:new] = true}
  max_iter.times do |iter|    
    was_change = explore_subsets(bounds, ref_set, max_no_improv, step_size)
    ref_set.sort!{|x,y| x[:cost] <=> y[:cost]}
    best = ref_set.first if ref_set.first[:cost] < best[:cost]
    #puts " > iter=#{(iter+1)}, best=#{best[:cost]}"
    break if !was_change
  end
  return best
end

:search

In [81]:
# problem configuration
problem_size = 3
bounds = Array.new(problem_size) {|i| [-5, +5]}

# algorithm configuration
max_iter = 100
step_size = (bounds[0][1]-bounds[0][0])*0.005
max_no_improv = 30
ref_set_size = 10
diverse_set_size = 20
no_elite = 5

# execute the algorithm
best = search(bounds, max_iter, ref_set_size, diverse_set_size, max_no_improv, step_size, no_elite)
puts "Done. Best Solution:","c=#{best[:cost]}","v=#{best[:vector].inspect}"

Done. Best Solution:
c=6.058258507310004e-07
v=[0.00032054347349045557, -0.0006040007758185386, 0.0003718343652007942]


## Stochastic Hill Climbing

In [82]:
def onemax(vector)
  return vector.inject(0.0){|sum, v| sum + ((v=="1") ? 1 : 0)}
end

:onemax

In [83]:
def random_bitstring(num_bits)
  return Array.new(num_bits){|i| (rand<0.5) ? "1" : "0"}
end

:random_bitstring

In [84]:
def random_neighbor(bitstring)  
  mutant = Array.new(bitstring)
  pos = rand(bitstring.size)
  mutant[pos] = (mutant[pos]=='1') ? '0' : '1'
  return mutant
end

:random_neighbor

In [85]:
def search(max_iterations, num_bits)
  candidate = {}
  candidate[:vector] = random_bitstring(num_bits)
  candidate[:cost] = onemax(candidate[:vector])
  max_iterations.times do |iter|
    neighbor = {}
    neighbor[:vector] = random_neighbor(candidate[:vector])
    neighbor[:cost] = onemax(neighbor[:vector])
    candidate = neighbor if neighbor[:cost] >= candidate[:cost]
    #puts " > iteration #{(iter+1)}, best=#{candidate[:cost]}"
    break if candidate[:cost] == num_bits
  end 
  return candidate
end

:search

In [86]:
# problem configuration
num_bits = 64

# algorithm configuration
max_iterations = 1000

# execute the algorithm
best = search(max_iterations, num_bits)
puts "Done. Best Solution:","c=#{best[:cost]}","v=#{best[:vector].join}"

Done. Best Solution:
c=64.0
v=1111111111111111111111111111111111111111111111111111111111111111


## Tabu Search

In [87]:
def euc_2d(c1, c2)
  Math.sqrt((c1[0] - c2[0])**2.0 + (c1[1] - c2[1])**2.0).round
end

:euc_2d

In [88]:
def cost(perm, cities)
  distance = 0
  perm.each_with_index do |c1, i|
    c2 = (i==perm.size-1) ? perm[0] : perm[i+1]
    distance += euc_2d(cities[c1], cities[c2])
  end
  return distance
end

:cost

In [89]:
def random_permutation(cities)
  perm = Array.new(cities.size){|i| i}
  perm.each_index do |i|
    r = rand(perm.size-i) + i
    perm[r], perm[i] = perm[i], perm[r]
  end
  return perm
end

:random_permutation

In [90]:
def stochastic_two_opt(parent)
  perm = Array.new(parent)
  c1, c2 = rand(perm.size), rand(perm.size)
  exclude = [c1]
  exclude << ((c1==0) ? perm.size-1 : c1-1)
  exclude << ((c1==perm.size-1) ? 0 : c1+1)
  c2 = rand(perm.size) while exclude.include?(c2)
  c1, c2 = c2, c1 if c2 < c1
  perm[c1...c2] = perm[c1...c2].reverse
  return perm, [[parent[c1-1], parent[c1]], [parent[c2-1], parent[c2]]]
end

:stochastic_two_opt

In [91]:
def is_tabu?(permutation, tabu_list)
  permutation.each_with_index do |c1, i|
    c2 = (i==permutation.size-1) ? permutation[0] : permutation[i+1]
    tabu_list.each do |forbidden_edge|
      return true if forbidden_edge == [c1, c2]
    end
  end
  return false
end

:is_tabu?

In [92]:
def generate_candidate(best, tabu_list, cities)
  perm, edges = nil, nil
  begin
    perm, edges = stochastic_two_opt(best[:vector])
  end while is_tabu?(perm, tabu_list)  
  candidate = {:vector=>perm}
  candidate[:cost] = cost(candidate[:vector], cities)
  return candidate, edges
end

:generate_candidate

In [93]:
def search(cities, tabu_list_size, candidate_list_size, max_iter)
  current = {:vector=>random_permutation(cities)}
  current[:cost] = cost(current[:vector], cities)
  best = current
  tabu_list = Array.new(tabu_list_size)
  max_iter.times do |iter|
    candidates = Array.new(candidate_list_size) do |i|
      generate_candidate(current, tabu_list, cities)
    end
    candidates.sort! {|x,y| x.first[:cost] <=> y.first[:cost]}
    best_candidate = candidates.first[0]
    best_candidate_edges = candidates.first[1]
    if best_candidate[:cost] < current[:cost]
      current = best_candidate
      best = best_candidate if best_candidate[:cost] < best[:cost]
      best_candidate_edges.each {|edge| tabu_list.push(edge)}
      tabu_list.pop while tabu_list.size > tabu_list_size
    end
    #puts " > iteration #{(iter+1)}, best=#{best[:cost]}"
  end
  return best
end


:search

In [94]:
# problem configuration
# algorithm configuration
max_iter = 100
tabu_list_size = 15
max_candidates = 50

# execute the algorithm
best = search(berlin52, tabu_list_size, max_candidates, max_iter)
puts "Done. Best Solution:","c=#{best[:cost]}","v=#{best[:vector].inspect}"


Done. Best Solution:
c=8473
v=[5, 42, 44, 18, 40, 7, 8, 9, 32, 50, 10, 11, 27, 25, 26, 12, 51, 13, 46, 15, 28, 49, 19, 22, 29, 41, 1, 6, 20, 16, 2, 17, 30, 21, 0, 31, 48, 35, 34, 33, 43, 45, 36, 38, 39, 37, 4, 14, 23, 47, 24, 3]


## Variable Neighborhood Search

In [95]:
def euc_2d(c1, c2)
  Math.sqrt((c1[0] - c2[0])**2.0 + (c1[1] - c2[1])**2.0).round
end

:euc_2d

In [96]:
def cost(perm, cities)
  distance =0
  perm.each_with_index do |c1, i|
    c2 = (i==perm.size-1) ? perm[0] : perm[i+1]
    distance += euc_2d(cities[c1], cities[c2])
  end
  return distance
end

:cost

In [97]:
def random_permutation(cities)
  perm = Array.new(cities.size){|i| i}
  perm.each_index do |i|
    r = rand(perm.size-i) + i
    perm[r], perm[i] = perm[i], perm[r]
  end
  return perm
end

:random_permutation

In [98]:
def stochastic_two_opt!(perm)
  c1, c2 = rand(perm.size), rand(perm.size)
  exclude = [c1]
  exclude << ((c1==0) ? perm.size-1 : c1-1)
  exclude << ((c1==perm.size-1) ? 0 : c1+1)
  c2 = rand(perm.size) while exclude.include?(c2)
  c1, c2 = c2, c1 if c2 < c1
  perm[c1...c2] = perm[c1...c2].reverse
  return perm
end

:stochastic_two_opt!

In [99]:
def local_search(best, cities, max_no_improv, neighborhood)
  count = 0
  begin
    candidate = {}
    candidate[:vector] = Array.new(best[:vector])
    neighborhood.times{stochastic_two_opt!(candidate[:vector])}
    candidate[:cost] = cost(candidate[:vector], cities)
    if candidate[:cost] < best[:cost]
      count, best = 0, candidate
    else
      count += 1      
    end
  end until count >= max_no_improv
  return best
end

:local_search

In [102]:
def search(cities, neighborhoods, max_no_improv, max_no_improv_ls)
  best = {}
  best[:vector] = random_permutation(cities)
  best[:cost] = cost(best[:vector], cities)
  iter, count = 0, 0
  begin
    neighborhoods.each do |neigh|
      candidate = {}
      candidate[:vector] = Array.new(best[:vector])      
      neigh.times{stochastic_two_opt!(candidate[:vector])}
      candidate[:cost] = cost(candidate[:vector], cities)
      candidate = local_search(candidate, cities, max_no_improv_ls, neigh)      
      #puts " > iteration #{(iter+1)}, neigh=#{neigh}, best=#{best[:cost]}"
      iter += 1
      if(candidate[:cost] < best[:cost])
        best, count = candidate, 0
        #puts "New best, restarting neighborhood search."
        break
      else
        count += 1
      end
    end  
  end until count >= max_no_improv
  return best
end

:search

In [103]:
# problem configuration
# algorithm configuration
max_no_improv = 50
max_no_improv_ls = 70
neighborhoods = 1...20

# execute the algorithm
best = search(berlin52, neighborhoods, max_no_improv, max_no_improv_ls)
puts "Done. Best Solution:","c=#{best[:cost]}","v=#{best[:vector].inspect}"

Done. Best Solution:
c=9185
v=[3, 24, 11, 50, 10, 51, 13, 12, 46, 25, 26, 27, 47, 23, 37, 39, 4, 14, 42, 32, 9, 8, 7, 40, 18, 44, 2, 6, 1, 41, 20, 16, 29, 28, 15, 49, 19, 43, 45, 36, 33, 21, 0, 22, 30, 17, 31, 48, 34, 35, 38, 5]
